In [1]:
import pandas as pd
import requests
import zipfile
from io import BytesIO
from datetime import datetime
import gc

In [2]:
############################################################################################
def conexao(url_do_zip, max_tentativas = 50):
          
    for tentativa in range(max_tentativas):
        try:
            response = requests.get(url_do_zip, stream=True, timeout=0.3)
            response.raise_for_status()  # Verifica se houve algum erro no download
            print("Conexão aceita na {} tentativa".format(tentativa + 1))
            return response
        except requests.exceptions.RequestException as e:
            if tentativa+1 == max_tentativas:
                print("Erro reportado: {}", e)
    else:
        print("Falha ao fazer a conexão para o download do arquivo zip após {} tentativas.".format(max_tentativas))
        return ''
############################################################################################
def zip_dos_dados(response):
    
    if response.status_code == 200:
        zip_data = BytesIO(response.content)
        return zip_data
    else:
        print("Falha ao fazer o download do arquivo zip.")
        return ''
############################################################################################
def csv_dos_dados(zip_data, nome_do_arquivo_csv):

    with zipfile.ZipFile(zip_data, 'r') as zip_ref:
        for filename in zip_ref.namelist():
            if nome_do_arquivo_csv in filename and filename.endswith('.csv'):
                with zip_ref.open(filename) as csv_file:
                    df = pd.read_csv(csv_file, sep=';', encoding='utf-16', 
                                     dayfirst=True, engine='c',
                                     low_memory=False)
                    return df
        print("O arquivo CSV não foi encontrado no arquivo zip.")
        return ''

In [3]:
url_do_zip = 'https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR_Filtrado/Arquivos_csv_2020.zip'
nome_do_arquivo_csv = '_Pedidos_csv_'
response = conexao(url_do_zip)
zip_data = zip_dos_dados(response)
df = csv_dos_dados(zip_data, nome_do_arquivo_csv)
print(df.shape)
df = df.assign(Ano='2020')
df_consolidado = df.copy()
print(df_consolidado.shape)

Conexão aceita na 4 tentativa
(111363, 21)
(111363, 22)


In [4]:
url_do_zip = 'https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR_Filtrado/Arquivos_csv_2021.zip'
nome_do_arquivo_csv = '_Pedidos_csv_'
response = conexao(url_do_zip)
zip_data = zip_dos_dados(response)
df = csv_dos_dados(zip_data, nome_do_arquivo_csv)
print(df.shape)
df = df.assign(Ano='2021')
df_consolidado = pd.concat([df_consolidado, df], ignore_index=True)
print(df_consolidado.shape)

Conexão aceita na 1 tentativa
(79545, 21)
(190908, 22)


In [5]:
url_do_zip = 'https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR_Filtrado/Arquivos_csv_2022.zip'
nome_do_arquivo_csv = '_Pedidos_csv_'
response = conexao(url_do_zip)
zip_data = zip_dos_dados(response)
df = csv_dos_dados(zip_data, nome_do_arquivo_csv)
print(df.shape)
df = df.assign(Ano='2022')
df_consolidado = pd.concat([df_consolidado, df], ignore_index=True)
print(df_consolidado.shape)

Conexão aceita na 1 tentativa
(69504, 21)
(260412, 22)


In [6]:
df_filtrado = df_consolidado[(df_consolidado['Esfera'] == 'Federal') 
                  & (df_consolidado['Situacao'] == 'Concluída') 
                  & (df_consolidado['OrigemSolicitacao'] == 'Internet')]
print(df_filtrado.shape)

(259220, 22)


In [7]:
df_campos_alvo = df_filtrado[['OrgaoDestinatario', 'DataRegistro', 'DetalhamentoSolicitacao', 'Ano']]
contagem_ocorrencias = df_campos_alvo['OrgaoDestinatario'].value_counts()
contagem_acumulada = contagem_ocorrencias.cumsum()
total_ocorrencias = contagem_ocorrencias.sum()
limite = 0.85 * total_ocorrencias
registros_selecionados = contagem_acumulada[contagem_acumulada <= limite]

In [8]:
recorte_amostral = df_campos_alvo[df_campos_alvo['OrgaoDestinatario'].isin(registros_selecionados.index)].copy()
recorte_amostral['DataRegistro'] = pd.to_datetime(recorte_amostral['DataRegistro'], format='%d/%m/%Y')
recorte_amostral['Dia da Semana'] = recorte_amostral['DataRegistro'].dt.day_name()
recorte_amostral['Mês'] = recorte_amostral['DataRegistro'].dt.month_name()
print(recorte_amostral.shape)

(220185, 6)


In [9]:
recorte_amostral.to_pickle('df_pedidos_baixados_2020_a_2023.pkl')